In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import os, sys
import numpy as np

In [2]:
dataset = np.loadtxt('data.csv', delimiter=',')

In [3]:
def createmodel():
    ilayer = layers.Dense(units = 7, input_dim = len(dataset[0,:-4]), activation="relu")
    layer1 = layers.Dense(units = 8, activation="relu")
    layer2 = layers.Dense(units = 9, activation="relu")
    olayer = layers.Dense(units = 4, activation = "sigmoid")

    Model = keras.Sequential([ilayer,layer1,layer2,olayer])
    
    Model.compile(
    optimizer = keras.optimizers.SGD(0.001),
    loss="binasry_crossentropy"
    )
    
    return Model

In [ ]:
''' Building the NN '''

tf.keras.backend.clear_session()

fraction = 0.35
point = int(len(dataset)*fraction)

xtrain = dataset[:point,:-4]
ytrain = dataset[:point,-4:]

xtest  = dataset[point:,:-4]
ytest  = dataset[point:,-4:]

model2 = createmodel()

#saving the model
modelfolder = '/Users/dan-ioanbultoc/Desktop/EE/training/'

os.path.dirname(modelfolder)
modelsaver = tf.keras.callbacks.ModelCheckpoint(filepath = modelfolder, save_weights_only = True)

''' Training the NN '''

history = model2.fit(x = xtrain, y=ytrain, epochs=10, verbose = False, callbacks = [modelsaver])
plt.plot(history.history["loss"])
plt.show()

2022-05-09 12:10:49.056318: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-09 12:10:49.062839: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-09 12:10:49.065283: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: AMD Radeon Pro 560X


2022-05-09 12:10:50.692006: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [ ]:
def process(prediction):#converts from % prediction to [0,0,1,0]
    for i in range(len(prediction)):
        Maxj = 0
        for j in range(len(prediction[i])):
            if prediction[i][j] > prediction[i][Maxj]:
                Maxj = j       
        prediction[i] = [0 if prediction[i][Maxj] > prediction[i][j] else 1 for j in range(len(prediction[i]))]

In [ ]:
def analyzemodel(model, xtestdata, ytestdata):
    pred = model.predict(xtestdata)
    process(pred)
    
    diff = abs(pred-ytestdata)
    
    eff = 0
    for predict in diff:
        eff += 1
        for feature in predict:
            if feature != 0:
                eff -= 1
                break
    
    eff /= len(diff)
    return eff*100

In [ ]:
print(analyzemodel(model2, xtrain, ytrain))

#print(ytrain[:50,:]) 

In [ ]:
savedmodel = createmodel()
savedmodel.load_weights(modelfolder)

In [ ]:
print(ytest[0])
print(pred[0])